# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
%load_ext dotenv
%dotenv 

In [2]:
import os
import sys
#from logger import get_logger
#_logs = get_logger(__name__)

import pandas as pd
import numpy as np
import os
from glob import glob
sys.path.append(os.getenv('SRC_DIR'))

In [3]:
from logger import get_logger
_logs = get_logger(__name__)

In [4]:
import dask.dataframe as dd

/var/folders/2r/1l6yc2bx3m56rxmdpbz1z5w80000gn/T/ipykernel_61344/676544213.py:1: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can be reported on the Dask issue tracker
https://github.com/dask/dask/issues 

  import dask.dataframe as dd


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [10]:

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
PRICE_DATA

parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet"))
parquet_files

['../../05_src/data/prices/AMZN/AMZN_2023.parquet',
 '../../05_src/data/prices/AMZN/AMZN_2015.parquet',
 '../../05_src/data/prices/AMZN/AMZN_2014.parquet',
 '../../05_src/data/prices/AMZN/AMZN_2022.parquet',
 '../../05_src/data/prices/AMZN/AMZN_2016.parquet',
 '../../05_src/data/prices/AMZN/AMZN_2020.parquet',
 '../../05_src/data/prices/AMZN/AMZN_2021.parquet',
 '../../05_src/data/prices/AMZN/AMZN_2017.parquet',
 '../../05_src/data/prices/AMZN/AMZN_2024.parquet',
 '../../05_src/data/prices/AMZN/AMZN_2013.parquet',
 '../../05_src/data/prices/AMZN/AMZN_2018.parquet',
 '../../05_src/data/prices/AMZN/AMZN_2019.parquet',
 '../../05_src/data/prices/RCL/RCL_2019.parquet',
 '../../05_src/data/prices/RCL/RCL_2018.parquet',
 '../../05_src/data/prices/RCL/RCL_2013.parquet',
 '../../05_src/data/prices/RCL/RCL_2024.parquet',
 '../../05_src/data/prices/RCL/RCL_2021.parquet',
 '../../05_src/data/prices/RCL/RCL_2017.parquet',
 '../../05_src/data/prices/RCL/RCL_2016.parquet',
 '../../05_src/data/prices

In [11]:

dd_px = dd.read_parquet(parquet_files).set_index("ticker")
dd_px.compute()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year
ticker,,,,,,,,,
A,2024-01-02 00:00:00-05:00,137.491402,139.879263,137.212819,138.048569,1441600,0.0,0.0,2024
A,2024-01-03 00:00:00-05:00,137.302366,137.302366,130.407407,130.496948,2074500,0.0,0.0,2024
A,2024-01-04 00:00:00-05:00,129.890031,130.835226,129.531851,130.337753,2446600,0.0,0.0,2024
A,2024-01-05 00:00:00-05:00,129.342827,131.292926,127.969799,129.899994,1394000,0.0,0.0,2024
A,2024-01-08 00:00:00-05:00,129.482108,132.894776,129.153774,132.705734,1311400,0.0,0.0,2024
...,...,...,...,...,...,...,...,...,...
YUM,2024-01-25 00:00:00-05:00,127.913548,128.386458,126.642582,128.297791,1871700,0.0,0.0,2024
YUM,2024-01-26 00:00:00-05:00,128.002213,128.760851,126.770665,127.184464,1157000,0.0,0.0,2024
YUM,2024-01-29 00:00:00-05:00,127.194317,128.622910,126.396261,128.622910,1482100,0.0,0.0,2024


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [31]:
# Write your code below.
dd_feat = (dd_px.groupby('ticker', group_keys=False)
           .apply(
                    lambda x: x.assign(Close_lag = x['Close'].shift(1))
                ).assign(
                    Adj_close = lambda x: (1 - (x['Dividends']/ x['Close'])) * x['Close']
                ).assign(
                    hi_lo_range = lambda x: (x['High']- x['Low'])
                ).assign(
                     Adj_close_lag = lambda x: ((1 - (x['Dividends']/ x['Close'])) * x['Close']).shift(1)
                ).assign(
                    returns = lambda x: x['Adj_close']/x['Adj_close_lag'] - 1
                )
                
                )
                


/var/folders/2r/1l6yc2bx3m56rxmdpbz1z5w80000gn/T/ipykernel_61344/3457114861.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False)


In [32]:
dd_feat.compute()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,Close_lag,Adj_close,hi_lo_range,Adj_close_lag,returns
ticker,,,,,,,,,,,,,,
A,2024-01-02 00:00:00-05:00,137.491402,139.879263,137.212819,138.048569,1441600,0.0,0.0,2024,NaN,138.048569,2.666444,NaN,NaN
A,2024-01-03 00:00:00-05:00,137.302366,137.302366,130.407407,130.496948,2074500,0.0,0.0,2024,138.048569,130.496948,6.894959,138.048569,-0.054703
A,2024-01-04 00:00:00-05:00,129.890031,130.835226,129.531851,130.337753,2446600,0.0,0.0,2024,130.496948,130.337753,1.303375,130.496948,-0.001220
A,2024-01-05 00:00:00-05:00,129.342827,131.292926,127.969799,129.899994,1394000,0.0,0.0,2024,130.337753,129.899994,3.323127,130.337753,-0.003359
A,2024-01-08 00:00:00-05:00,129.482108,132.894776,129.153774,132.705734,1311400,0.0,0.0,2024,129.899994,132.705734,3.741002,129.899994,0.021599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,2024-01-25 00:00:00-05:00,127.913548,128.386458,126.642582,128.297791,1871700,0.0,0.0,2024,127.529305,128.297791,1.743877,127.529305,0.006026
YUM,2024-01-26 00:00:00-05:00,128.002213,128.760851,126.770665,127.184464,1157000,0.0,0.0,2024,128.297791,127.184464,1.990186,128.297791,-0.008678
YUM,2024-01-29 00:00:00-05:00,127.194317,128.622910,126.396261,128.622910,1482100,0.0,0.0,2024,127.184464,128.622910,2.226649,127.184464,0.011310


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [39]:
# Write your code below.


dd_feat['rolling_10_mean'] = dd_feat['returns'].rolling(10).mean()

dd_feat.compute()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,Close_lag,Adj_close,hi_lo_range,Adj_close_lag,returns,rolling_10_mean
ticker,,,,,,,,,,,,,,,
A,2024-01-02 00:00:00-05:00,137.491402,139.879263,137.212819,138.048569,1441600,0.0,0.0,2024,NaN,138.048569,2.666444,NaN,NaN,NaN
A,2024-01-03 00:00:00-05:00,137.302366,137.302366,130.407407,130.496948,2074500,0.0,0.0,2024,138.048569,130.496948,6.894959,138.048569,-0.054703,NaN
A,2024-01-04 00:00:00-05:00,129.890031,130.835226,129.531851,130.337753,2446600,0.0,0.0,2024,130.496948,130.337753,1.303375,130.496948,-0.001220,NaN
A,2024-01-05 00:00:00-05:00,129.342827,131.292926,127.969799,129.899994,1394000,0.0,0.0,2024,130.337753,129.899994,3.323127,130.337753,-0.003359,NaN
A,2024-01-08 00:00:00-05:00,129.482108,132.894776,129.153774,132.705734,1311400,0.0,0.0,2024,129.899994,132.705734,3.741002,129.899994,0.021599,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,2024-01-25 00:00:00-05:00,127.913548,128.386458,126.642582,128.297791,1871700,0.0,0.0,2024,127.529305,128.297791,1.743877,127.529305,0.006026,0.000828
YUM,2024-01-26 00:00:00-05:00,128.002213,128.760851,126.770665,127.184464,1157000,0.0,0.0,2024,128.297791,127.184464,1.990186,128.297791,-0.008678,0.000115
YUM,2024-01-29 00:00:00-05:00,127.194317,128.622910,126.396261,128.622910,1482100,0.0,0.0,2024,127.184464,128.622910,2.226649,127.184464,0.011310,0.001145


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

calculating rolling means can also be done in Dask according to dask documentation, and since dask is better at handling very large dataframes, it may have been better to do so in dask. 

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.